In [ ]:
import sys, os
sys.path.append(os.path.abspath('..')) # add parent folder to path
import pandas_ta as pta
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
datapath = '../data/GOLD.csv'
df = pd.read_csv(datapath, index_col=2)
df.index = pd.to_datetime(df.index)
df


In [ ]:
def get_Most_Short_Comodity(datapath, start_date, end_date):
    df = pd.read_csv(datapath)
    selected_df = df.loc[(df['Report_Date_as_YYYY-MM-DD'] >= start_date) & (df['Report_Date_as_YYYY-MM-DD'] <= end_date)]
    grouped_df = selected_df.groupby('Report_Date_as_YYYY-MM-DD')['Pct_of_OI_M_Money_Short_All'].max()
    grouped_df = grouped_df.to_frame()
    ret = pd.merge(grouped_df,df, on=['Report_Date_as_YYYY-MM-DD', 'Pct_of_OI_M_Money_Short_All'] , how='left')[['Report_Date_as_YYYY-MM-DD', 'Market_and_Exchange_Names', 'Pct_of_OI_M_Money_Short_All']]
    # print(ret)
    return ret

In [ ]:
def get_low_short_Comodities(datapath, start_date, end_date):
    df = pd.read_csv(datapath)
    selected_df = df.loc[(df['Report_Date_as_YYYY-MM-DD'] >= start_date) & (df['Report_Date_as_YYYY-MM-DD'] <= end_date)]
    
    selected_df['Report_Date_as_YYYY-MM-DD'] = pd.to_datetime(selected_df['Report_Date_as_YYYY-MM-DD'])
    selected_df.set_index('Report_Date_as_YYYY-MM-DD', inplace=True)
    
    grouped_df = selected_df.groupby('Market_and_Exchange_Names')
    
    frames = []  # To store the dataframes we're interested in
    
    for name, group in grouped_df:
        group['Rolling_Mean'] = group['Pct_of_OI_M_Money_Short_All'].rolling(20).mean()
        group['Rolling_Std'] = group['Pct_of_OI_M_Money_Short_All'].rolling(20).std()
        group = group[group['Pct_of_OI_M_Money_Short_All'] > (group['Rolling_Mean'] + 2 * group['Rolling_Std']) or \
                      group['Pct_of_OI_M_Money_Short_All'] < (group['Rolling_Mean'] - 2 * group['Rolling_Std'])]
        frames.append(group)
        
    # Concatenate all the dataframes we're interested in
    result = pd.concat(frames)
    
    # Sort the result by date
    result = result.sort_index()
    
    # Save the result to a CSV file
    result.to_csv('../data/tempcsv.csv')
    
    # Concatenate all the dataframes we're interested in




In [13]:
def get_most_outstanding_short_Comodities(datapath, start_date, end_date):
    
    df = pd.read_csv(datapath)
    selected_df = df.loc[(df['Report_Date_as_YYYY-MM-DD'] >= start_date) & (df['Report_Date_as_YYYY-MM-DD'] <= end_date)]
    
    selected_df['Report_Date_as_YYYY-MM-DD'] = pd.to_datetime(selected_df['Report_Date_as_YYYY-MM-DD'])
    selected_df.set_index('Report_Date_as_YYYY-MM-DD', inplace=True)
    
    grouped_df = selected_df.groupby('Market_and_Exchange_Names')
    
    frames = []  # To store the dataframes we're interested in
    
    for name, group in grouped_df:
        group['Rolling_Mean'] = group['Pct_of_OI_M_Money_Short_All'].rolling(20).mean()
        group['Rolling_Std'] = group['Pct_of_OI_M_Money_Short_All'].rolling(20).std()
        group = group[abs(group['Pct_of_OI_M_Money_Short_All'] - group['Rolling_Mean']) > 2 * group['Rolling_Std']]
        group['Commodity'] = name  # Add the commodity name to the DataFrame
        frames.append(group)
        
    # Concatenate all the dataframes we're interested in
    result = pd.concat(frames)
    
    # Sort the result by date
    result = result.sort_index()

    # Get the commodity with max standard deviation for each day
    result['Excess'] = (result['Pct_of_OI_M_Money_Short_All'] - result['Rolling_Mean']) / result['Rolling_Std']
    
    result = result.reset_index()  # Reset the index so 'Report_Date_as_YYYY-MM-DD' is a column again

    most_shorted_each_day = result.loc[result.groupby('Report_Date_as_YYYY-MM-DD')['Excess'].idxmax()][['Report_Date_as_YYYY-MM-DD', 'Market_and_Exchange_Names', 'CFTC_Market_Code_Quotes', 'CFTC_Contract_Market_Code_Quotes', 'FutOnly_or_Combined', 'Excess']]
    least_shorted_each_day = result.loc[result.groupby('Report_Date_as_YYYY-MM-DD')['Excess'].idxmin()][['Report_Date_as_YYYY-MM-DD', 'Market_and_Exchange_Names', 'CFTC_Market_Code_Quotes', 'CFTC_Contract_Market_Code_Quotes', 'FutOnly_or_Combined', 'Excess']]
    
    # Save the result to a CSV file
    most_shorted_each_day.to_csv('../data/most_shorted_each_day.csv', index=False)
    least_shorted_each_day.to_csv('../data/least_shorted_each_day.csv', index=False)


In [14]:
test = get_most_outstanding_short_Comodities(datapath, '2019-01-01', '2023-12-31')

C:\Users\lqs\AppData\Local\Temp\ipykernel_7060\3069874011.py:3: DtypeWarning: Columns (120,121,126,133,134,135,140,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datapath)
C:\Users\lqs\AppData\Local\Temp\ipykernel_7060\3069874011.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Commodity'] = name  # Add the commodity name to the DataFrame
C:\Users\lqs\AppData\Local\Temp\ipykernel_7060\3069874011.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [ ]:
test = get_low_short_Comodities(datapath, '2020-01-01', '2022-12-31')

In [ ]:
ret = get_Most_Short_Comodity(datapath, '2019-01-01', '2022-12-31')
ret.to_csv('../data/Most_Short_Comodity.csv', index=False)